In [2]:
import os

from typing import Any, Mapping, List, Tuple, Dict

import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from loguru import logger
from datetime import datetime
from itertools import chain

import torch
from torch import nn
from torch.nn.init import constant_, kaiming_normal_
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import pack_sequence, unpack_sequence

In [3]:
BASE_DIR = "/Users/artemvopilov/Programming/yandex_cup_2023"

In [4]:
DATA_DIR = f"{BASE_DIR}/data"

TRAIN_DF_PATH = f"{DATA_DIR}/train.csv"
TEST_DF_PATH = f"{DATA_DIR}/test.csv"

NORMED_EMBEDDINGS_DIR = f"{BASE_DIR}/normed_embeddings"
PCA_EMBEDDINGS_DIR = f"{BASE_DIR}/pca_embeddings"
VAE_EMBEDDINGS_DIR = f"{BASE_DIR}/vae_embeddings"
NORMED_LSTM_EMBEDDINGS_DIR = f"{BASE_DIR}/normed_lstm_embeddings"
VAE_LSTM_EMBEDDINGS_DIR = f"{BASE_DIR}/vae_lstm_embeddings"

In [16]:
DEVICE = "cpu"

TAGS_N = 256

INPUT_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = TAGS_N

EPOCHS = 10

### Read data

In [6]:
train_df = pd.read_csv(TRAIN_DF_PATH)
test_df = pd.read_csv(TEST_DF_PATH)

In [7]:
track_id_to_embeddings = {}
for fn in tqdm(os.listdir(VAE_EMBEDDINGS_DIR)):
    fp = f"{VAE_EMBEDDINGS_DIR}/{fn}"

    track_id = fn.split('.')[0]
    embeddings = np.load(fp).astype(np.float32)
    track_id_to_embeddings[track_id] = embeddings

  0%|          | 0/76714 [00:00<?, ?it/s]

### Dataset

In [8]:
class LSTMDataset(Dataset):
    def __init__(self, df: pd.DataFrame, track_id_to_embeddings: Dict[str, np.ndarray[np.float64]], tags_n: int, is_testing=False):
        self._df = df
        self._track_id_to_embeddings = track_id_to_embeddings
        self._tags_n = tags_n
        self._is_testing = is_testing
        
    def __len__(self):
        return self._df.shape[0]

    def __getitem__(self, index: int) -> Tuple[str, np.ndarray[np.float64], List[np.ndarray[np.int64]]]:
        row = self._df.iloc[index]
        track_id = row["track"]
        embeddings = self._track_id_to_embeddings[str(track_id)]
        if self._is_testing:
            return track_id, embeddings, np.array([])
        tags = [int(x) for x in row["tags"].split(',')]
        target = np.zeros(self._tags_n)
        target[tags] = 1
        return track_id, embeddings, target

In [9]:
def collate_fn(b):
    track_ids = torch.from_numpy(np.vstack([x[0] for x in b]))
    embeddings = [torch.from_numpy(x[1]) for x in b]
    targets = torch.from_numpy(np.vstack([x[2] for x in b]))
    return track_ids, embeddings, targets

### Model

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.apply(self._init_layer)

    def forward(self, embeddings):
        embeddings = pack_sequence(embeddings, enforce_sorted=False)
        cell_states, (final_hidden_state, final_cell_state) = self.lstm(embeddings)
        first_layer_hidden_state = final_hidden_state[0, :, :]
        return self.fc(first_layer_hidden_state), unpack_sequence(cell_states)

    def reset(self) -> None:
        self.apply(self._init_layer)

    @staticmethod
    def _init_layer(layer: nn.Module) -> None:
        if isinstance(layer, nn.Linear):
            kaiming_normal_(layer.weight.data)
            if layer.bias is not None:
                constant_(layer.bias.data, 0)

### Trainer

In [11]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = None
    alpha = 0.8
    for iteration, data in enumerate(loader):
        optimizer.zero_grad()
        _, embeddings, target = data
        
        embeddings = [x.to(DEVICE) for x in embeddings]
        target = target.to(DEVICE)
        
        predictions, _ = model(embeddings)
        
        loss = criterion(predictions, target)
        loss.backward()
        optimizer.step()
        
        if running_loss is None:
            running_loss = loss.item()
        else:
            running_loss = alpha * loss.item() + (1 - alpha) * loss.item()
        if iteration % 100 == 0:
            logger.info("{} batch {} loss {}".format(datetime.now(), iteration + 1, running_loss))

### Predictor

In [13]:
def predict(model, loader):
    model.eval()
    track_ids = []
    predictions = []
    cell_states = []
    with torch.no_grad():
        for data in loader:
            track_id, embeddings, _ = data
            
            embeddings =  [x.to(DEVICE) for x in embeddings]
            
            batch_predictions, batch_cell_states = model(embeddings)

            track_ids.append(track_id.numpy())
            predictions.append(batch_predictions.detach().cpu().numpy())
            cell_states.append([cs.detach().cpu().numpy() for cs in batch_cell_states])
    track_ids = np.vstack(track_ids).ravel()
    predictions = np.vstack(predictions)
    cell_states = list(chain.from_iterable(cell_states))
    return track_ids, predictions, cell_states

### Train

In [17]:
train_dataset = LSTMDataset(train_df, track_id_to_embeddings, TAGS_N, False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [18]:
model = LSTMModel(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.CrossEntropyLoss()

model = model.to(DEVICE)
criterion = criterion.to(DEVICE)
optimizer = Adam(model.parameters())

for epoch in tqdm(range(EPOCHS)):
    train_epoch(model, train_loader, criterion, optimizer)

  0%|          | 0/10 [00:00<?, ?it/s]

2023-11-12 10:49:18.657 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:49:18.657900 batch 1 loss 21.46189323812723
2023-11-12 10:49:57.720 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:49:57.720021 batch 101 loss 18.576463600620627
2023-11-12 10:50:39.105 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:50:39.105831 batch 201 loss 17.581225898116827
2023-11-12 10:51:22.394 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:51:22.394743 batch 301 loss 16.930438693612814
2023-11-12 10:52:03.372 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:52:03.372186 batch 401 loss 16.94636393431574
2023-11-12 10:52:43.525 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:52:43.525861 batch 501 loss 16.147415263578296
2023-11-12 10:53:26.611 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:53:26.611269 batch 601 loss 14.61594226770103
2023-11-12 10:54:06.681 | INFO     | __main__:train_epoch:23 - 2023-11-12 10:54:06.681739 batch 701 loss 17.365219809114933
2023-11-12 10

### Inference

In [19]:
inference_df = pd.concat([train_df, test_df], ignore_index=True)

In [20]:
inference_df.head()

,track,tags
0,49734,"5,6,9,26,32,55,96"
1,67845,"6,9,28,39,145,155"
2,25302,"0,6,28,40,116,168"
3,57796,"28,186"
4,13676,"6,23,177"


In [21]:
inference_df.shape

(76714, 2)

In [22]:
inference_dataset = LSTMDataset(inference_df, track_id_to_embeddings, TAGS_N, True)
inference_loader = DataLoader(inference_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [23]:
inference_track_ids, inference_predictions, inference_cell_states = predict(model, inference_loader)

In [24]:
len(inference_track_ids), len(inference_predictions), len(inference_cell_states)

(76714, 76714, 76714)

### Save predictions

In [25]:
predictions_df = pd.DataFrame([
    {'track': track, 'prediction': ','.join([str(p) for p in probs])}
    for track, probs in 
    zip(inference_track_ids, inference_predictions)
])

In [26]:
predictions_df.head()

,track,prediction
0,49734,"3.8992786,3.7550733,4.485249,3.7612824,1.31616..."
1,67845,"3.9249747,2.7431135,3.5394266,3.8076763,0.7991..."
2,25302,"3.7137194,3.2424319,3.0893915,3.0597296,1.3846..."
3,57796,"5.4363995,1.5872772,4.2686834,5.607044,1.63424..."
4,13676,"3.5169446,2.976041,3.4261286,2.5848532,0.59583..."


In [27]:
predictions_df.shape

(76714, 2)

In [28]:
predictions_df.to_csv('prediction_lstm_vae_2.csv', index=False)

### Save embeddings

In [29]:
track_id_to_lstm_embedding = {}

In [30]:
os.mkdir(VAE_LSTM_EMBEDDINGS_DIR)

In [31]:
for ti, embeddings in tqdm(zip(inference_track_ids, inference_cell_states)):
    fn = f"{ti}.npy"
    fp = f"{VAE_LSTM_EMBEDDINGS_DIR}/{fn}"
    np.save(fp, embeddings)

0it [00:00, ?it/s]